In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, MultipleLocator, LogLocator, LogFormatter, NullFormatter

In [ ]:
# ─── 6) Initialize histories ─────────────────────────────────────────────────
gen_hist     = {}
price_hist   = {}
reserve_hist = {}
bm_price     = {}
bm_gen       = {}
mg_by_hr     = {}
bm_gen_hourly = {}

In [ ]:
# ─── 7) Main intraday dispatch loop ──────────────────────────────────────────
for idx, year in enumerate(year_range):
    mean_h       = Elec_demand[idx] / 8760.0
    load_profile = load_frac * mean_h

    supply = np.zeros_like(load_profile)
    by_src = {}

    # 7.1 stack nondispatchable
    for src in nondisp:
        share = share_history.get(src, np.zeros_like(year_range))[idx]
        arr   = np.array([profile_funcs[src](h, share * mean_h) for h in hours])
        supply += arr
        by_src[src] = arr.copy()

    # 7.2 dispatchable “lump‐and‐slice”
    residual = np.maximum(load_profile - supply, 0.0)
    full_req = {s: share_history[s][idx] * mean_h * 24 for s in disp}

    for src in disp:
        req    = full_req[src]
        unmet  = list(np.where(residual > 0)[0])
        while req > 0 and unmet:
            per_h = req / len(unmet)
            alloc = np.zeros_like(load_profile)
            for h in unmet:
                alloc[h] = min(per_h, residual[h])
            placed     = alloc.sum()
            req       -= placed
            supply    += alloc
            by_src.setdefault(src, np.zeros_like(load_profile))
            by_src[src] += alloc
            residual   = np.maximum(residual - alloc, 0.0)
            unmet      = list(np.where(residual > 0)[0])

    # 7.3 lost load
    lost            = np.maximum(load_profile - supply, 0.0)
    by_src[LOLE_LABEL] = lost
    reserve_hist[year] = lost

    # 7.4 daily summary
    print(f"\n--- Year {year}: Daily Generation Summary ---")
    print(f"{'Source':<20}{'Actual (GWh)':>15}{'Target (GWh)':>15}{'Error (GWh)':>15}")
    for src, arr in by_src.items():
        actual = arr.sum()
        target = (share_history[src][idx] * mean_h * 24
                  if src != LOLE_LABEL else 0.0)
        print(f"{src:<20}{actual:15.4f}{target:15.4f}{(actual-target):15.4f}")

    # 7.5 marginal price & marginal plant
    price       = np.zeros_like(load_profile)
    mg          = np.empty(load_profile.shape, dtype=object)
    unmet_after = np.maximum(load_profile - supply, 0.0)

    for h_ix in range(len(hours)):
        if unmet_after[h_ix] > 0:
            mg[h_ix]    = LOLE_LABEL
            price[h_ix] = VoLL
        else:
            cum = 0.0
            for src in legend_order:
                if src == LOLE_LABEL:
                    continue
                cum += by_src.get(src, np.zeros_like(load_profile))[h_ix]
                if cum >= load_profile[h_ix]:
                    mg[h_ix]    = src
                    price[h_ix] = marginal_cost_table.get(src, VoLL)
                    break

    # 7.6 record histories
    gen_hist[year]   = pd.DataFrame(by_src, index=hours).T
    price_hist[year] = price
    bm_i             = np.argmin(load_profile)
    bm_price[year]   = price[bm_i]
    bm_gen[year]     = mg[bm_i]
    mg_by_hr[year]   = mg
    # hourly baseload plant
    bm_gen_hourly[year] = np.where(price == bm_price[year], mg, None)

    # ─── 8) Plot ───────────────────────────────────────────────────────────────
   # ─── Compact intraday outlook plot ────────────────────────────────────────────
    fig, (ax1, ax2) = plt.subplots(
        nrows=2,
        sharex=True,
        figsize=(3.5, 3),               
        gridspec_kw={"height_ratios":[1,1], "hspace":0.3}
    )
    fig.subplots_adjust(right=0.68, top=0.92, hspace=0.35)

    # Top: stacked supply vs. load
    bottom = np.zeros_like(load_profile)
    for src in legend_order:
        arr = by_src.get(src, np.zeros_like(load_profile))
        ax1.bar(
            hours, arr, bottom=bottom,
            color=source_colors[src],
            edgecolor="black" if src == LOLE_LABEL else None,
            label=src
        )
        bottom += arr
    ax1.plot(hours, load_profile, "k--", lw=1.5, label="Load")
    ax1.set_ylabel(r"$\mathbf{Consumption}$ [GW]", fontfamily="Arial", fontsize=9)
    ax1.set_title(f"Intraday Outlook {year}", fontfamily="Arial", fontsize=9, fontweight="bold")
    ax1.yaxis.set_major_locator(MultipleLocator(15))
    ax1.tick_params(axis='y', which='major', length=4, labelsize=7)
    ax1.set_xticks(np.arange(4, hours.max()+1, 4))
    ax1.margins(x=0, y=0)
    ax1.legend(
        loc="center left",
        bbox_to_anchor=(1.02, 0.5),
        ncol=2,
        prop={"family":"Arial", "size":7},
        frameon=False,
        handlelength=1,
        handletextpad=0.3,
        columnspacing=0.5,
        borderaxespad=0
    )

# Bottom: prices
    ax2.axhline(
        1e6 * VoLL,
        linestyle="-.",
        linewidth=0.8,
        label="VoLL",
        color=schemecolors["Dark Red"]
    )
    ax2.plot(
        hours, 1e6 * price,
        marker="o", markersize=3, linestyle="--",
        label="Price",
        color=schemecolors["Dark Blue"]
    )
    bm_lcoe_hr = 1e6 * np.array([marginal_cost_table.get(src, VoLL) for src in bm_gen_hourly[year]])
    ax2.set_ylabel(r"$\mathbf{Value}$ [USD/MWh]", fontfamily="Arial", fontsize=9)
    ax2.set_xticks(np.arange(4, hours.max()+1, 4))
    ax2.yaxis.set_major_locator(LogLocator(base=10, numticks=5))
    ax2.grid(axis="y", linestyle=":", linewidth=0.5)
    ax2.set_ylim(100,2000)
    ax2.set_yscale('log')
    ax2.legend(
        loc="upper left",
        bbox_to_anchor=(1.0, 1.0),
        prop= {"family":"Arial", "size":7},
        frameon=False
    )
    ax2.margins(x=0, y=0)
    ax2.grid(True)


    ax1.grid(True, which='major', axis='y', linestyle='-', linewidth=0.5)
    ax2.grid(True, which='major', axis='y', linestyle='-', linewidth=0.5)

    os.makedirs(out_dir, exist_ok=True)
    plt.savefig(os.path.join(out_dir, f"intraday_outlook_{year}_{scenario}.png"), dpi=300, bbox_inches="tight")
    plt.show()
    plt.close(fig)